In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import datasets
import imageio
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

%matplotlib inline

## Perceptron de uma camada

In [ ]:
class Perceptron(object):
   def __init__(self, rate = 0.01, niter = 10):
      self.rate = rate
      self.niter = niter

   def fit(self, X, y):
      # Pesos
      self.weight = np.zeros(1 + X.shape[1])

      # Número de classificações incorretas
      self.errors = []

      for i in range(self.niter):
         err = 0
         for xi, target in zip(X, y):
            delta_w = self.rate * (target - self.predict(xi))
            self.weight[1:] += delta_w * xi
            self.weight[0] += delta_w
            err += int(delta_w != 0.0)
         self.errors.append(err)
      return self

   def net_input(self, X):
      # Calcule a entrada líquida
      return np.dot(X, self.weight[1:]) + self.weight[0]

   def predict(self, X):
      # Devolver a etiqueta da classe após a etapa da unidade
      return np.where(self.net_input(X) >= 0.0, 1, -1)

#### Pré-processamento do conjunto de dados da Iris

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

In [ ]:
df.tail()

#### Extraindo os primeiros 100 rótulos de classe que correspondem às 50 flores Iris-Setosa e 50 flores Iris-Versicolor

In [ ]:
df.iloc[145:150, 0:5]

In [ ]:
y = df.iloc[0:100, 4].values
y

#### convertendo os rótulos de classe em dois rótulos de classe inteiros 1 Versicolor e -1 *Setosa*

In [ ]:
y = np.where(y == 'Iris-setosa', -1, 1)
y

#### Extração da primeira coluna de recursos (comprimento da sépala) e a terceira coluna de recursos (comprimento da pétala) das 100 amostras de treinamento

In [ ]:
X = df.iloc[0:100, [0, 2]].values
X

#### Visualização do gráfico de dispersão

In [ ]:
plt.scatter(X[:50, 0], X[:50, 1], color='red', marker='o', label='Iris-setosa')
plt.scatter(X[50:100, 0], X[50:100, 1], color='blue', marker='x', label='Iris-versicolor')
plt.xlabel('petal length')
plt.ylabel('sepal length')
plt.legend(loc='upper left')
plt.show()

#### Treinamento do modelo perceptron

In [ ]:
pn = Perceptron(0.1, 10)
pn.fit(X, y)
plt.plot(range(1, len(pn.errors) + 1), pn.errors, marker='o')
plt.xlabel('Epocas')
plt.ylabel('Número de classificações incorretas')
plt.show()

#### Visualize os limites de decisão

In [ ]:
def plot_decision_boundaries(X, y, classifier):
    h = .02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    s
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, palette={-1: 'red', 1: 'blue'}, legend='full', markers=['o', 'x'])
    plt.contourf(xx, yy, Z, alpha=0.2, cmap='coolwarm')
    plt.xlabel('Largura da Sépala [cm]')
    plt.ylabel('Largura da Pétala [cm]')
    plt.title('Limites de Decisão')
    plt.show()

plot_decision_boundaries(X, y, pn)

### Limites de Decisão com Plotly

In [ ]:

def plot_decision_boundaries(X, y, classifier):
    h = .02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    df = pd.DataFrame({'sepal length': xx.ravel(), 'petal length': yy.ravel(), 'class': Z.ravel()})

    fig = px.scatter(df, x='petal length', y='sepal length', color='class',
                     title='Limites de Decisão com Plotly', opacity=0.1,
                     color_continuous_scale=px.colors.diverging.RdYlBu)

    df_data = pd.DataFrame({'sepal length': X[:, 0], 'petal length': X[:, 1], 'class': y})
    fig.add_trace(px.scatter(df_data, x='petal length', y='sepal length', color='class').data[0])

    fig.show()

plot_decision_boundaries(X, y, pn)

#### Limites de Decisão com Plotly Projeção 3d

In [ ]:
def plot_decision_boundaries_3d(X, y, classifier):
    h = .02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)


    df = pd.DataFrame({'sepal length': xx.ravel(), 'petal length': yy.ravel(), 'class': Z.ravel()})


    fig = px.scatter_3d(df, x='petal length', y='sepal length', z='class', color='class',
                         title='Limites de Decisão', opacity=0,
                         color_continuous_scale=px.colors.diverging.RdYlBu)

    df_data = pd.DataFrame({'sepal length': X[:, 0], 'petal length': X[:, 1], 'class': y})
    fig.add_trace(px.scatter_3d(df_data, x='petal length', y='sepal length', z='class', color='class').data[0])

    fig.show()

plot_decision_boundaries_3d(X, y, pn)